## Read input data and turn it into data frame

In [8]:
# Dependancies
import csv
import os
import pandas as pd 

In [9]:
# Supporting functions
def get_filename(ticker):
    return os.path.join("..", "InputData", ticker + ".csv")

In [10]:
# Read input data into dataframe
def read_ticker(file_name):
    # Input file into dataframe
    rawdata = pd.read_csv(file_name) 
    # Date into date type, date of week, week of year, Gain
    rawdata.Date = rawdata.Date.astype('datetime64[ns]')
    # isocalendar reports end of the year as 1st week - see 12/31/2008
    #rawdata['YearWeek'] = [ td.year + td.isocalendar()[1] / 100 for td in rawdata.Date] 
    rawdata['YearWeek'] = [ td.year + (int(td.strftime("%W")) + 1) / 100 for td in rawdata.Date]
    rawdata['Weekday'] = [ td.weekday() for td in rawdata.Date]
    rawdata['GainPct'] = (rawdata.Close - rawdata.Open) / rawdata.Open
    
    # Pivot data into new df
    return rawdata.pivot(index='YearWeek', columns='Weekday', values='GainPct')

In [11]:
# Sum data
def sum_ticker(ticker):
    df = read_ticker(get_filename(ticker))
    result = pd.DataFrame([df.sum(axis = 0, skipna = True) * 100])
    result.insert(0, "Ticker", ticker)
    return result

In [ ]:
# Sum tickers
tickers = ["XLB","XLC","XLE","XLF","XLI","XLK","XLP","XLRE", "XLU", "XLV", "XLY"]

df = pd.DataFrame()

for ticker in tickers:
    print(ticker)
    df = df.append(sum_ticker(ticker))

df

XLB
XLC
XLE
XLF
XLI
XLK
XLP
XLRE
XLU
XLV


In [ ]:
# Total sum
total = pd.DataFrame([df.sum(axis = 0, skipna = True)])
total = total.drop(['Ticker'], axis=1)
total

In [ ]:
# Add column with maximum gain of the week
middata['MaxGainDay'] = middata.idxmax(axis=1)
middata.head()

In [ ]:
# Count days with highest weekly gains / minimal loses
#middata.groupby('MaxGainDay').size()

In [ ]:
# Sum gains total in %
